# Qwen/Qwen2.5-0.5B-Instruct

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}



In [3]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # the output is not tokenized, we have just text
    add_generation_prompt=True, # generation promp = <|im_start|>assistant at the end od the prompt
)
print(text)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print("model_inputs: ", model_inputs.keys())
print("model_inputs.input_ids.shape: ", model_inputs.input_ids.shape)
print()

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

# now the generated_ids containts also the input prompt. We remove it
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
print("generated_ids[0][:5]: ", generated_ids[0][:5], "...")
print("generated_ids[0].shape: ", generated_ids[0].shape)

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("\nresponse:\n", response, sep="")

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant

model_inputs:  dict_keys(['input_ids', 'attention_mask'])
model_inputs.input_ids.shape:  torch.Size([1, 39])

generated_ids[0][:5]:  tensor([  32, 3460, 4128, 1614,  374], device='cuda:0') ...
generated_ids[0].shape:  torch.Size([116])

response:
A large language model is a type of artificial intelligence that can generate human-like text based on the input it receives. These models are designed to understand and respond to complex questions and provide detailed information in various fields such as natural language processing, machine translation, and chatbot development. Large language models have been trained on vast amounts of data, including books, articles, and videos, to enable them to produce coherent and contextually appropriate responses. They are often used in applications lik

# deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
### Usage Recommendations

We recommend adhering to the following configurations when utilizing the DeepSeek-R1 series models, including benchmarking, to achieve the expected performance:

1. Set the temperature within the range of 0.5-0.7 (0.6 is recommended) to prevent endless repetitions or incoherent outputs.
2. **Avoid adding a system prompt; all instructions should be contained within the user prompt.**
3. For mathematical problems, it is advisable to include a directive in your prompt such as: "Please reason step by step, and put your final answer within \boxed{}."
4. When evaluating model performance, it is recommended to conduct multiple tests and average the results.

Additionally, we have observed that the DeepSeek-R1 series models tend to bypass thinking pattern (i.e., outputting "<think>\n\n</think>") when responding to certain queries, which can adversely affect the model's performance. To ensure that the model engages in thorough reasoning, we recommend enforcing the model to initiate its response with "<think>\n" at the beginning of every output.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
model.generation_config.pad_token_id = tokenizer.pad_token_id

print(model.generation_config)

GenerationConfig {
  "bos_token_id": 151646,
  "do_sample": true,
  "eos_token_id": 151643,
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_p": 0.95
}



In [5]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True, # generation promp = <think>\n at the end od the prompt
)
print(text)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print("model_inputs: ", model_inputs.keys())
print("model_inputs.input_ids.shape: ", model_inputs.input_ids.shape)
print()

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
print("generated_ids[0][:5]: ", generated_ids[0][:5], "...")
print("generated_ids[0].shape: ", generated_ids[0].shape)

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("\nresponse:\n", response, sep="")

<｜begin▁of▁sentence｜><｜User｜>Give me a short introduction to large language model.<｜Assistant｜><think>

model_inputs:  dict_keys(['input_ids', 'attention_mask'])
model_inputs.input_ids.shape:  torch.Size([1, 16])

generated_ids[0][:5]:  tensor([32313,    11,   773,   358,  1184], device='cuda:0') ...
generated_ids[0].shape:  torch.Size([512])

response:
Okay, so I need to write a short introduction about Large Language Models. Hmm, where do I start? I know that LLMs are these big AI systems that can understand and generate human language. But what exactly are they? I think they're models that can process vast amounts of text data, right? They're used in various applications like chatbots, writing assistance, maybe even creative writing. 

Wait, how do they work? I remember they take in a lot of text data and use some kind of algorithm to generate new text. But I'm not entirely sure about the specifics. Maybe they learn from data, so they can generate coherent text based on that data. O

---

Trying 4bit quantization.

In [6]:
import time
prompt = "Write a story about a dragon and a knight of approximately 300 words."
messages = [
    {"role": "user", "content": prompt}
]
t0 = time.time()
for _ in range(10):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True, # generation promp = <think>\n at the end od the prompt
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

t1 = time.time()
print("Time required for 10 generations: ", round(t1 - t0, 2))

Time required for 10 generations:  84.65


In [7]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
    quantization_config=quantization_config
)
model_4bit.generation_config.pad_token_id = tokenizer.pad_token_id

t0 = time.time()
for _ in range(10):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True, # generation promp = <think>\n at the end od the prompt
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model_4bit.device)
    generated_ids = model_4bit.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

t1 = time.time()
print("Time required for 10 generations: ", round(t1 - t0, 2))

Time required for 10 generations:  96.1


In [8]:
print("memory footprint (in bytes):", model.get_memory_footprint(), end="\n")
print("memory footprint (in bytes):", model_4bit.get_memory_footprint())

memory footprint (in bytes): 3554176256
memory footprint (in bytes): 1588882688
